## Imports

In [6]:
from bs4 import BeautifulSoup
from time import sleep
from urllib.request import Request, urlopen

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from requests_html import HTMLSession
from playwright.sync_api import sync_playwright
from playwright.async_api import async_playwright

from tqdm import tqdm

import pandas as pd
import csv
import base64
from PIL import Image
from io import BytesIO
import os
import time
import requests
import traceback
import re

In [7]:
#------------------------------------------------- Image Extraction Function -------------------------------------------------#
# search for query in google and get the image
def get_image(query, page):
    print(f"searching for {query}")
    url = f"https://www.google.com/search?q={query}&tbm=isch"
    links = []
    # we have to get top 5 images
    page.goto(url)
    # to get high quality image, we load it first
    while True:
        try:
            page.click(".FRuiCf.islib.nfEiy")
            break
        except:
            time.sleep(0.5)
            continue
    time.sleep(1)
    x = 0
    while True:
        try:
            # once the image is loaded, we get the link
            link = page.query_selector(".sFlh5c.pT0Scc.iPVvYb").get_attribute("src")
            if link not in links:
                links.append(link)
            else:
                x += 1
                image_buttons = page.query_selector_all(".FRuiCf.islib.nfEiy")
                image_buttons[x].click()
                time.sleep(2)
                continue
            if len(links) >= 5:
                break
            x += 1
            image_buttons = page.query_selector_all(".FRuiCf.islib.nfEiy")
            image_buttons[x].click()
            time.sleep(2)
        except:
            x += 1
            image_buttons = page.query_selector_all(".FRuiCf.islib.nfEiy")
            image_buttons[x].click()
            time.sleep(2)
            continue

    return links  

In [8]:
#------------------------------------------------- Browser Session Function -------------------------------------------------#
def get_browser():
    # this will give us a session of the browser
    p = sync_playwright().start()
    path = os.path.join(os.getcwd(), "browser_session")
    browser = p.chromium.launch_persistent_context(path, headless=False)
    page = browser.new_page()
    return page

In [9]:
# Read in the file
images_df = pd.read_csv(r"/Users/kylewang/Desktop/MLSA/Scripts/Upwork/google_images.csv", encoding= 'unicode_escape')
images_df

,ï»¿HTL,Google_Search,Google_Image_Href_1,Google_Image_Href_2,Google_Image_Href_3,Google_Image_Href_4,Google_Image_Href_5
0,DA75201400.OLIVE.ST.................,SHERATON DALLAS 400 OLIVE ST,https://dynamic-media-cdn.tripadvisor.com/medi...,NaN,NaN,NaN,NaN
1,GR760511501.GAYLORD.TRL.............,GAYLORD TEXAS RESORT & CONF CTR 1501 GAYLORD TRL,https://assets.simpleviewinc.com/simpleview/im...,NaN,NaN,NaN,NaN
2,DA752072201.N.STEMMONS.FWY..........,HILTON ANATOLE HOTEL 2201 N STEMMONS FWY,https://assets.resortpass.com/uploads/image/pi...,NaN,NaN,NaN,NaN
3,HO770101600.LAMAR.ST................,HILTON AMERICAS CONVENTION CTR 1600 LAMAR ST,https://assets.simpleviewinc.com/simpleview/im...,NaN,NaN,NaN,NaN
4,DA75207300.E.REUNION.BLVD...........,HYATT REGENCY DALLAS DOWNTOWN 300 E REUNION BLVD,https://media-cdn.tripadvisor.com/media/photo-...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
7526,HO770707700.WILLOW.CHASE.BLVD.APT.17,MARRIOTT EXECUSTAY 7700 WILLOW CHASE BLVD APT ...,,,NaN,NaN,NaN
7527,DR786202390.PROCHNOW.RD.............,MT GAINOR INN 2390 PROCHNOW RD,,,NaN,NaN,NaN
7528,GO78629222.SAINT.PETER.ST...........,BELLE OAKS INN 222 SAINT PETER ST,,,NaN,NaN,NaN
7529,HO770247855.KATY.FWY................,CLSD WELLESLEY INN & SUITES 7855 KATY FWY,,,NaN,NaN,NaN


In [10]:
image_1 = []
image_2 = []
image_3 = []
image_4 = []
image_5 = []

async with async_playwright() as p:
    with tqdm(total=len(images_df)) as pbar:
        for index, row in images_df.iterrows():
            browser = await p.chromium.launch()
            page = await browser.new_page()
            query = row['Google_Search'].replace(" ", "+").replace("&", "%26")
            try:
                links = get_image(query, page)
            except:
                try:
                    page.close()
                except:
                    pass
                browser = await p.chromium.launch()
                page = await browser.new_page()
                links = get_image(query, page)
            try:
                image_1.append(links[0])
            except:
                image_1.append('')
            try:
                image_2.append(links[1])
            except:
                image_2.append('')
            try:
                image_3.append(links[3])
            except:
                image_3.append('')
            try:
                image_4.append(links[4])
            except:
                image_4.append('')
            try:
                image_5.append(links[5])
            except:
                image_5.append('')
            page.close()
            pbar.update(1)

  0%|                                                  | 0/7531 [00:00<?, ?it/s]

searching for SHERATON+DALLAS+400+OLIVE+ST


/var/folders/2p/mkvb6kbd08l30n0fx72jcwz80000gn/T/ipykernel_9431/2530748215.py:8: RuntimeWarning: coroutine 'Page.goto' was never awaited
  page.goto(url)
/var/folders/2p/mkvb6kbd08l30n0fx72jcwz80000gn/T/ipykernel_9431/2530748215.py:12: RuntimeWarning: coroutine 'Page.click' was never awaited
  page.click(".FRuiCf.islib.nfEiy")
/var/folders/2p/mkvb6kbd08l30n0fx72jcwz80000gn/T/ipykernel_9431/127078054.py:17: RuntimeWarning: coroutine 'Page.close' was never awaited
  page.close()


searching for SHERATON+DALLAS+400+OLIVE+ST


  0%|                                                  | 0/7531 [00:02<?, ?it/s]


TypeError: 'coroutine' object is not subscriptable